# Torchvision Object Detection Finetuning Tutorial
---

* [Mask R-CNN](https://arxiv.org/abs/1703.06870)
* [Penn-Fudan Database for Pedestrian Detection and Segmentation](https://www.cis.upenn.edu/~jshi/ped_html/)

# Defining the Dataset
---

Supports adding new custom datasets.

Only specificity is that the dataset `__getitem__` should return:
* image: a PIL Image of size `(H, W)`
* target: a dict containing the following fields
    - `boxes (FloatTensor[N, 4])`: the coordinates of the `N` bounding boxes in `[x0, y0, x1, y1]` format, ranging from `0` to `W` and `0` to `H`
    - `labels (Int64Tensor[N])`: the label for each bounding box. `0` representss always the background class.
    - `image_id (Int64Tensor[1])`: an image identifier. It should be unique between all the images in the dataset, and is used during evaluation
    - `area (Tensor[N])`: The area of the bounding box. This is used during evaluation with the COCO metric, to separate the metric scores between small, medium and large boxes.
    - `iscrowd (UInt8Tensor[N])`: instances with iscrowd=True will be ignored during evaluation.
    - (optionally) `masks (UInt8Tensor[N, H, W])`: The segmentation masks for each one of the objects
    - (optionally) `keypoints (FloatTensor[N, K, 3])`: For each one of the N objects, it contains the K keypoints in `[x, y, visibility]` format, defining the object. visibility=0 means that keypoints is not visible. Note that for data augmentation, the notion of flipping a keypoint is dependant on the data representation, and you should probably adapt `references/detection/transforms.py` for your new keypoint representation

## <u>Writing a custom dataset for PennFudan</u>

<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image01.png">

<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image02.png">

In [1]:
import os
import numpy as np
import torch
from PIL import Image

class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        #   ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
    
    def __getitem__(self, idx):
        # load images and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        #   because each color corresponds to a different instance
        #   with 0 being the background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        #   of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        
        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        return img, target
    
    def __len__(self):
        return len(self.imgs)

# Defining your model
---

<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image03.png">

Mask R-CNN adds an extra branch into Faster R-CNN, which also predicts segmentation masks for each instance.

<img src="https://pytorch.org/tutorials/_static/img/tv_tutorial/tv_image04.png">

There are two common situations where one might want to modify one of the available models in torchvision modelzoo. The first is when we want to start from a pre-trained model, and just finetune the last layer. The other is when we want to replace the backbone of the model with a different one (for faster predictions, for example).

## <u>1 - Finetuning from a pretrained model</u>

In [2]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

# replace the classifier with a new one, that has
#   num_classes which is user-defined
num_classes = 2 # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/pha/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


## <u>2 - Modifying the model to add a different backbone</u>

In [1]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
#   only the features
backbone = torchvision.models.mobilenet_v2(weights="DEFAULT").features
# FasterRCNN needs to know the number of
#   output channels in a backbone. For mobilenet_v2, it's 1280
#   so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
#   location, with 5 different sizes and 3 different aspect
#   ratios. We have a Tuple[Tuple[int]] because each feature
#   map could potentially have different sizes and aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),), 
                                    aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
#   use to perform the region of interest cropping, as well as
#   the size of the crop after rescaling.
#   if your backbone returns a Tensor, featmap_names is expected to
#   be [0]. More generally, the backbone should return an
#   OrderedDict[Tensor], and in featmap_name you can choose which
#   feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/blast/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

## <u>An Instance segmentation model for PennFudan Dataset</u>

In our case, we want to fine-tune from a pre-trained model, given that our dataset is very small, so we will be following approach number 1.

In [18]:
# Here we want to also compute the instance segmentation masks, 
#   so we will be using Mask R-CNN:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # loadn an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model


# Putting Everything Together
---

In [9]:
# helper functions for data augmentation and transformation
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# Testing `forward()` method (Optional)
---

In [13]:
import utils

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
dataset = PennFudanDataset('../data/PennFudanPed', get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn
)
# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets) # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)


In [22]:
# main function
from engine import train_one_epoch, evaluate
import utils

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('../data/PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('../data/PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation dataloaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn
    )

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn
    )

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)
    
    # move model to the right device
    model.to(device)

    # construct and optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3,
                                                    gamma=0.1)

    # epoch
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)
    
    print("Done")
    

In [23]:
main()

Epoch: [0]  [ 0/60]  eta: 0:00:16  lr: 0.000090  loss: 4.3397 (4.3397)  loss_classifier: 0.6401 (0.6401)  loss_box_reg: 0.2397 (0.2397)  loss_mask: 3.4125 (3.4125)  loss_objectness: 0.0400 (0.0400)  loss_rpn_box_reg: 0.0074 (0.0074)  time: 0.2719  data: 0.1486  max mem: 3874
Epoch: [0]  [10/60]  eta: 0:00:07  lr: 0.000936  loss: 1.9152 (2.5533)  loss_classifier: 0.4869 (0.4494)  loss_box_reg: 0.2787 (0.3035)  loss_mask: 1.4281 (1.7609)  loss_objectness: 0.0293 (0.0337)  loss_rpn_box_reg: 0.0063 (0.0059)  time: 0.1557  data: 0.0154  max mem: 3874
Epoch: [0]  [20/60]  eta: 0:00:05  lr: 0.001783  loss: 1.0246 (1.6961)  loss_classifier: 0.1963 (0.3100)  loss_box_reg: 0.2427 (0.2673)  loss_mask: 0.4332 (1.0814)  loss_objectness: 0.0227 (0.0297)  loss_rpn_box_reg: 0.0057 (0.0078)  time: 0.1388  data: 0.0022  max mem: 3874
Epoch: [0]  [30/60]  eta: 0:00:04  lr: 0.002629  loss: 0.6494 (1.3414)  loss_classifier: 0.1252 (0.2438)  loss_box_reg: 0.2117 (0.2515)  loss_mask: 0.2541 (0.8134)  loss_ob